In [449]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow import keras
from sklearn.metrics import mean_squared_error

In [437]:
# import temperature data
data = pd.read_csv('temp/temperature.csv').values
data = np.delete(data,0,1)
print(data)
n_steps = 4

[[27.37 23.29]
 [28.77 20.87]
 [28.73 22.41]
 [28.13 20.35]
 [28.37 19.5 ]
 [29.07 17.68]
 [28.53 19.27]
 [27.7  19.13]
 [27.07 20.2 ]
 [28.27 21.39]
 [28.13 20.72]
 [27.33 20.3 ]
 [27.1  21.56]
 [28.53 22.15]
 [28.33 23.65]
 [27.6  24.13]
 [28.3  23.85]
 [28.77 22.79]
 [29.03 24.22]
 [27.6  23.43]
 [28.07 24.24]
 [28.63 23.02]
 [27.9  19.6 ]
 [27.57 21.4 ]
 [27.07 20.8 ]
 [28.47 22.6 ]
 [28.17 22.63]
 [27.93 22.63]]


In [438]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    x, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        x.append(seq_x)
        y.append(seq_y)
    return np.array(x), np.array(y)

In [439]:

# convert into input/output
x, y = split_sequences(data, n_steps)


# print(data[20:23])
for i in range(len(x)):
	print(x[i], y[i])
n_features=x.shape[2]
# X.shape
data.shape
print(x.shape[2])

[[27.37 23.29]
 [28.77 20.87]
 [28.73 22.41]
 [28.13 20.35]] [28.37 19.5 ]
[[28.77 20.87]
 [28.73 22.41]
 [28.13 20.35]
 [28.37 19.5 ]] [29.07 17.68]
[[28.73 22.41]
 [28.13 20.35]
 [28.37 19.5 ]
 [29.07 17.68]] [28.53 19.27]
[[28.13 20.35]
 [28.37 19.5 ]
 [29.07 17.68]
 [28.53 19.27]] [27.7  19.13]
[[28.37 19.5 ]
 [29.07 17.68]
 [28.53 19.27]
 [27.7  19.13]] [27.07 20.2 ]
[[29.07 17.68]
 [28.53 19.27]
 [27.7  19.13]
 [27.07 20.2 ]] [28.27 21.39]
[[28.53 19.27]
 [27.7  19.13]
 [27.07 20.2 ]
 [28.27 21.39]] [28.13 20.72]
[[27.7  19.13]
 [27.07 20.2 ]
 [28.27 21.39]
 [28.13 20.72]] [27.33 20.3 ]
[[27.07 20.2 ]
 [28.27 21.39]
 [28.13 20.72]
 [27.33 20.3 ]] [27.1  21.56]
[[28.27 21.39]
 [28.13 20.72]
 [27.33 20.3 ]
 [27.1  21.56]] [28.53 22.15]
[[28.13 20.72]
 [27.33 20.3 ]
 [27.1  21.56]
 [28.53 22.15]] [28.33 23.65]
[[27.33 20.3 ]
 [27.1  21.56]
 [28.53 22.15]
 [28.33 23.65]] [27.6  24.13]
[[27.1  21.56]
 [28.53 22.15]
 [28.33 23.65]
 [27.6  24.13]] [28.3  23.85]
[[28.53 22.15]
 [28.33 23

In [440]:
model = Sequential()
model.add(LSTM(100,return_sequences=True, input_shape=(n_steps, n_features) ))
model.add(LSTM(100))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')

In [441]:
# fit model
model.fit(x,y,epochs=1000, verbose=0)
# demonstrate prediction

In [442]:
xinput=np.array([[27.37,23.29]
,[28.77,20.87]
,[28.73,22.41]
,[28.13,20.35]])
xinput = xinput.reshape((1, n_steps, n_features))
temp = model.predict(xinput, verbose= 0)
print(temp)


[[28.064434 21.70397 ]]


In [443]:
newModel = Sequential()
newModel.add(LSTM(100,return_sequences=True,batch_input_shape=(1,4,2),stateful=True))
newModel.add(LSTM(100))
newModel.add(Dense(n_features))
newModel.compile(optimizer='adam', loss='mean_squared_error')
newModel.set_weights(model.get_weights())

In [444]:
newtemp = x[-1]
print(newtemp)
pp=np.delete(newtemp, obj=[0,0,1])
pp=pp.flatten()
pp=np.append(pp, values = temp.flatten())
print(pp)
pp.shape
pp=pp.reshape((1, n_steps, n_features))
print(pp)



[[27.57 21.4 ]
 [27.07 20.8 ]
 [28.47 22.6 ]
 [28.17 22.63]]
[27.07       20.8        28.47       22.6        28.17       22.63
 28.06443405 21.70396996]
[[[27.07       20.8       ]
  [28.47       22.6       ]
  [28.17       22.63      ]
  [28.06443405 21.70396996]]]


In [445]:
predict_the_future = newModel.predict(pp, verbose=0)



In [446]:
tt = np.copy(predict_the_future)
print(tt)
for i in range(8):
    pp=np.delete(newtemp, obj=[0,0,1])
    pp=pp.flatten()
    pp=np.append(pp, values = predict_the_future.flatten())
    pp=pp.reshape((1, n_steps, n_features))
    # print(pp)
    predict_the_future = newModel.predict(pp, verbose=0)
    tt = np.append(tt,predict_the_future)
    print(tt)
    newModel.reset_states()


[[28.064049 21.703669]]
[28.064049 21.703669 28.074104 21.711342]
[28.064049 21.703669 28.074104 21.711342 28.064049 21.703669]
[28.064049 21.703669 28.074104 21.711342 28.064049 21.703669 28.064049
 21.703669]
[28.064049 21.703669 28.074104 21.711342 28.064049 21.703669 28.064049
 21.703669 28.064049 21.703669]
[28.064049 21.703669 28.074104 21.711342 28.064049 21.703669 28.064049
 21.703669 28.064049 21.703669 28.064049 21.703669]
[28.064049 21.703669 28.074104 21.711342 28.064049 21.703669 28.064049
 21.703669 28.064049 21.703669 28.064049 21.703669 28.064049 21.703669]
[28.064049 21.703669 28.074104 21.711342 28.064049 21.703669 28.064049
 21.703669 28.064049 21.703669 28.064049 21.703669 28.064049 21.703669
 28.064049 21.703669]
[28.064049 21.703669 28.074104 21.711342 28.064049 21.703669 28.064049
 21.703669 28.064049 21.703669 28.064049 21.703669 28.064049 21.703669
 28.064049 21.703669 28.064049 21.703669]


In [447]:
tt = tt.flatten()
print(tt)


[28.064049 21.703669 28.074104 21.711342 28.064049 21.703669 28.064049
 21.703669 28.064049 21.703669 28.064049 21.703669 28.064049 21.703669
 28.064049 21.703669 28.064049 21.703669]
28.064049


heelo
